<a href="https://colab.research.google.com/github/jscore95/apicustom/blob/main/Falcon_7B_Instruct2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer
from datasets import load_dataset

# Load Quantized Model

In [ ]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

pretrained_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

pretrained_model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(pretrained_model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, config)
print_trainable_parameters(peft_model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Prepare Data

In [ ]:
# Load dataset
def generate_prompt(data_point):
  return f"""
<Human>: {data_point["description"]}
<AI>: {data_point["title"]}
  """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, truncation=True, padding=True)
  return tokenized_full_prompt

# data = load_dataset("csv", data_files="fakelang.csv")
data = load_dataset("csv", data_files="mrbeast-title-generator.csv")

train_dataset = data['train'].shuffle().map(generate_and_tokenize_prompt)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

{'title': "I Bought The World's Largest Firework ($600,000)",
 'description': "title for a video about purchasing and setting off the world's largest firework.",
 'Unnamed: 2': None,
 'Unnamed: 3': None,
 'input_ids': [39,
  23431,
  48190,
  4120,
  312,
  241,
  2028,
  544,
  9992,
  273,
  4438,
  597,
  248,
  1079,
  18,
  94,
  5141,
  2858,
  1451,
  25,
  193,
  39,
  17362,
  48190,
  295,
  341,
  1165,
  390,
  2889,
  18,
  94,
  57823,
  5848,
  1451,
  204,
  3866,
  6549,
  23,
  1321,
  20],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

# Example Before Fine Tuning

In [ ]:
def generate(index):
  example_text = "title for a video about where people need to keep standing to win $500,000"
  correct_answer = train_dataset[index]['title']

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = peft_model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=100, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print("Best Answer:")
  print(correct_answer)

  print()

In [ ]:
generate(0)

# Training

In [ ]:
trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        auto_find_batch_size=True,
        num_train_epochs=10,
        # per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        # max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.863600
2,3.799200
3,3.703500
4,3.719000
5,3.560200
6,3.530300
7,3.814800
8,3.441700
9,3.155500
10,3.047600


TrainOutput(global_step=30, training_loss=2.6476991693178813, metrics={'train_runtime': 82.6336, 'train_samples_per_second': 11.981, 'train_steps_per_second': 0.363, 'total_flos': 797513126292480.0, 'train_loss': 2.6476991693178813, 'epoch': 9.23})

# Example After Fine Tuning

In [ ]:
peft_model.config.use_cache = True
peft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear4bit(in_

In [ ]:
generate(0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
title for a video about where people need to keep standing to win $500,000
Answer:
title for a video about where people need to keep standing to win $500,000.
$500,000 Standing Challenge - Who Can Keep Standing For 5 Minutes?
Best Answer:
I Bought The World's Largest Firework ($600,000)



# Connect to HF Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Merge Adapters and Save Model to Hub

In [ ]:
peft_model.save_pretrained("falcon-7b-3", push_to_hub=True) # save LORA adapters
peft_model.push_to_hub("jscore2023/falcon-7b-3")

# Create a new merged model

In [ ]:
from transformers import AutoModelForCausalLM, PretrainedConfig
import torch

# reload the base model
base_model = AutoModelForCausalLM.from_pretrained("vilsonrodrigues/falcon-7b-instruct-sharded", device_map={"":0}, trust_remote_code=True, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel

# load perf model with new adapters
merged_model = PeftModel.from_pretrained(
    base_model,
    "jscore2023/falcon-7b-3",
)

In [ ]:
merged_model = merged_model.merge_and_unload() # merge adapters with base model

In [ ]:
merged_model.push_to_hub("jscore2023/falcon-7b-3") # takes too long

# Push tokenizers to the hub

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vilsonrodrigues/falcon-7b-instruct-sharded", trust_remote_code=True)
tokenizer.push_to_hub("jscore2023/falcon-7b-3", use_auth_token=True)

# Testing our new model

In [ ]:
from typing import Dict, Any
import logging

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModel
import torch.cuda

LOGGER = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
device = "cuda" if torch.cuda.is_available() else "cpu"

# config = PeftConfig.from_pretrained("jscore2023/falcon-7b-instruct-ft-adapters")
model = AutoModelForCausalLM.from_pretrained("vilsonrodrigues/falcon-7b-instruct-sharded", device_map={"":0}, trust_remote_code=True, torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, "jscore2023/falcon-7b-instruct-ft-adapters")
# model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained("vilsonrodrigues/falcon-7b-instruct-sharded")

In [ ]:
model.eval()

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
encoding = tokenizer("title for a video about a biking endurance challenge with a million-dollar prize.", return_tensors="pt").to("cuda:0")


In [ ]:
encoding

{'input_ids': tensor([[18234, 15874,  1843,  7653,   312,   241,  1106,    24,   523, 48081,
           953, 20483,  5432]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=100, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
output

tensor([[18234, 15874,  1843,  7653,   312,   241,  1106,    24,   523, 48081,
           953, 20483,  5432,   193,   487,  3217,  1056,  8126,   597,   248,
         12122,   269,  5420,   275,   241,   916,    24,   366,  7080,   269,
           953,   345,   334, 13325,   788,   248, 53333,  8873,    23, 40349,
           431,   241,  2122,   275,  1026, 35299, 29619,    25,    11]],
       device='cuda:0')

In [ ]:
tokenizer.decode(output[0])